In [1]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage
# from dotenv import load_dotenv
# load_dotenv()

if not os.environ.get("ANTHROPIC_API_KEY"):
      os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter API key for Anthropic: ")


model = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")


model.invoke("Hello, vibhaw!")

AIMessage(content="I want to clarify that I'm Claude, an AI assistant created by Anthropic, not Vibhaw. I aim to be direct and honest about who and what I am. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_015p3GZXC4EbyQ3DtWXpGBCb', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 49, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--b473c5be-0664-441e-9d9b-94e058b7f648-0', usage_metadata={'input_tokens': 13, 'output_tokens': 49, 'total_tokens': 62, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [2]:
##create message 
messsgaes = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="what are two benefits of using  langchain"),]
model.invoke(messsgaes)

AIMessage(content='Here are two key benefits of using LangChain:\n\n1. Chain Composition and Modularity:\n- LangChain allows you to easily combine multiple AI operations into chains of tasks\n- You can break down complex workflows into smaller, reusable components\n- Makes it simpler to create sophisticated AI applications by connecting different tools and models in a modular way\n\n2. Built-in Abstractions and Integrations:\n- Provides pre-built components for common operations like memory management, document loading, and text splitting\n- Offers ready-to-use integrations with various LLMs (like GPT-3, GPT-4) and vector stores\n- Reduces boilerplate code and speeds up development by providing consistent interfaces for different AI services\n\nThese features make LangChain particularly valuable for building complex AI applications while keeping the code organized and maintainable.', additional_kwargs={}, response_metadata={'id': 'msg_01Smvmf6aAqGpoQkEvb3oJY9', 'model': 'claude-3-5-son

In [3]:
for chunk in model.stream(messsgaes):
    print(chunk.content, end="", flush=True)

Here are two key benefits of using LangChain:

1. Chain Management and Composition:
- LangChain makes it easy to combine multiple AI operations into sequential chains
- You can create complex workflows by connecting different components (prompts, models, memory, etc.)
- Helps organize and manage the flow of data between different parts of your AI application

2. Built-in Abstractions and Integrations:
- Provides pre-built components for common tasks (document loading, text splitting, vector stores, etc.)
- Offers integrations with multiple LLM providers (OpenAI, Hugging Face, etc.)
- Reduces boilerplate code and makes it easier to switch between different tools and services

These features make LangChain particularly useful for building complex LLM-powered applications while keeping the code organized and maintainable.

In [4]:
##dynamic prompt template
from langchain.prompts import ChatPromptTemplate

##create translation app


translation_template=ChatPromptTemplate.from_messages(
    [
       ( "system",
        "You are a helpful assistant that translates {source_language} to {target_language}.maintain the tone and style",),
        ("user",
        "Translate the following text from {source_language} to {target_language}: {text}")
    ]
)

##
prompt=translation_template.invoke({
    "source_language": "English",
    "target_language": "French",
    "text": "lanchchain is a framework for developing applications powered by language models."
})


In [5]:
model.invoke(prompt)
for chunk in model.stream(prompt):
    print(chunk.content, end="", flush=True)

La langchain est un cadre de développement pour créer des applications alimentées par des modèles de langage.

Building First Chain


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def create_story_chain():
    ##template for story generation
    story_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a story generator. You will be given a topic and given theme, and you will write a short story about that topic with that tone."),
        ("user", "Write a story about {topic} with the theme of {theme}. /n Settings: {settings}"),
        ]    
    )


    ##template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages(
        [   
            ("system", "You are a story analyzer. You will be given a story and you will analyze it."),
            ("user", "Analyze the following story: {story}"),
        ]
    )


    story_chain=(
        story_prompt | model | StrOutputParser() 
    )

    ##craete a function to pass the story to analysis 
    def analyze_story(story):
        return {
            "story": story
        }

    analysis_chain=(
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain

In [10]:
chain= create_story_chain()
chain

ChatPromptTemplate(input_variables=['settings', 'theme', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a story generator. You will be given a topic and given theme, and you will write a short story about that topic with that tone.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['settings', 'theme', 'topic'], input_types={}, partial_variables={}, template='Write a story about {topic} with the theme of {theme}. /n Settings: {settings}'), additional_kwargs={})])
| ChatAnthropic(model='claude-3-5-sonnet-latest', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate

In [13]:
result=chain.invoke({
    "topic": "indian culture",
    "theme": "celebration",   
    "settings": "festivals, traditions, and family gatherings",
})
print(result)

Here's my analysis of "Colors of Love":

Theme:
The primary themes are:
- Cultural preservation and inheritance
- Family bonds and traditions
- Intergenerational connections
- The importance of cultural identity
- Love and unity

Setting:
- An Indian household during Diwali celebrations
- The story moves between kitchen, veranda, and courtyard
- Rich sensory details create an immersive atmosphere

Characters:
- Priya: 15-year-old protagonist, representing the new generation learning traditions
- Dadi (grandmother): The family matriarch, keeper of traditions
- Extended family members, including cousins, aunts, and uncles
- Each character represents different generations coming together

Literary Elements:
1. Symbolism:
- Diyas represent hope and enlightenment
- Food preparation symbolizes passing down of cultural knowledge
- The silk saree tradition represents coming of age and cultural continuity

2. Sensory Details:
- Aromatic descriptions (cardamom, saffron)
- Visual imagery (rangoli